### Colocalization-based ion image clusternig with DeepION

Publication: "DeepION: A Deep Learning-Based Low-Dimensional Representation Model of Ion Images for Mass Spectrometry Imaging" by Lei Guo. Analytical Chemistry. 2024. https://pubs.acs.org/doi/full/10.1021/acs.analchem.3c05002

Code by Lei Guo: https://github.com/gankLei-X/DeepION/tree/main

In [1]:
import os
import time
import pickle
import torch
import numpy as np
import pandas as pd
from itertools import chain
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import HDBSCAN
import random
import umap

# Import necessary Moran Imaging modules
from moran_imaging.ari_balance import balanced_adjusted_rand_index
from moran_imaging.vmeasure_balance import balanced_homogeneity_completeness_v_measure

# Import necessary DeepION module
from moran_imaging.deep_ion_clustering import DeepION_training, DeepION_predicting

# Fix random seed
random.seed(0)

C:\Users\Leo\anaconda3\envs\spatial_IMS_env_compare\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Verify GPU availability
print(torch.cuda.is_available())  
print(torch.cuda.device_count())  
print(torch.cuda.get_device_name(0))  
print("Torch CUDA Version:", torch.version.cuda)
print("Torch Backend CUDA Available:", torch.backends.cudnn.enabled)

# Set CUDA device 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

True
1
Quadro RTX 5000
Torch CUDA Version: 12.4
Torch Backend CUDA Available: True


In [3]:
def make_ion_image(data, image_shape, background_mask, fill_zeros=True, show_image=True, colormap='coolwarm'):
    
    # Fill background pixels with zeros (default) or NaN
    pixel_grid = np.zeros((image_shape[0]*image_shape[1], ))
    pixel_grid[np.invert(background_mask)] = data
    
    if not fill_zeros:
        pixel_grid[background_mask] = np.nan
    
    # Reshape data
    ion_image = np.reshape(pixel_grid, image_shape)
    
    # Plot ion image 
    if show_image == True: 
        plt.figure(dpi=100)
        plt.imshow(ion_image, cmap=colormap)
        plt.axis('off')
        plt.show()
        
    return ion_image

In [4]:
# Function to train DeepION model
def train_deepion(input_matrix, input_shape, mode="COL"):
    """
    Runs DeepION training with the given MSI dataset.
    
    Parameters:
        input_matrix (str): Path to the MSI data matrix file.
        input_shape (tuple): Shape of the MSI dataset (height, width).
        mode (str): Mode of operation, either 'COL' (colocalization) or 'ISO' (isotope discovery).
    """
    print("Starting DeepION training...")
    DeepION_training(input_matrix, input_shape, mode)
    print("Training completed!")

# Function to run DeepION feature extraction
def predict_deepion(input_matrix, input_shape, mode="COL"):
    """
    Runs DeepION feature extraction on the given MSI dataset.
    
    Parameters:
        input_matrix (str): Path to the MSI data matrix file.
        input_shape (tuple): Shape of the MSI dataset (height, width).
        mode (str): Mode of operation, either 'COL' (colocalization) or 'ISO' (isotope discovery).
    
    Returns:
        np.ndarray: Extracted feature matrix.
    """
    print("Extracting features using DeepION...")
    features = DeepION_predicting(input_matrix, input_shape, mode)
    print("Feature extraction completed!")
    return features

# Function to apply dimensionality reduction
def reduce_dimensionality(features):
    """
    Applies dimensionality reduction to extracted features.
    
    Parameters:
        features (np.ndarray): Feature matrix extracted from DeepION.
    
    Returns:
        np.ndarray: Low-dimensional feature representation.
    """
    print("Applying dimensionality reduction...")

    features_umap = umap.UMAP(n_components=20, metric='cosine', random_state=0).fit_transform(features)
    features_umap = MinMaxScaler().fit_transform(features_umap)
    
    print("Dimensionality reduction completed!")
    return features_umap

# Function to run co-localized ion searching
def run_colocalized_search(ld_features, peak_list, num=5, output_file="output/"):
    """
    Runs co-localized ion searching using DeepION features.
    
    Parameters:
        ld_features (np.ndarray): Low-dimensional feature representation.
        peak_list (str): Path to MSI peak list file.
        num (int): Number of co-localized ions to search for each ion.
        output_file (str): Path to save results.
    """
    print(f"Running co-localized ion searching (Top {num} ions)...")
    Co_localizedIONSearching(ld_features, peak_list, num, output_file)
    print(f"Results saved to {output_file}")

#### Transform zebra-fish dataset into a DeepION-compatible format

In [5]:
# Load dataset
with open('Zebra_fish_8_clusters_dataset.pickle', 'rb') as file:
    clustering_data_metadata = pickle.load(file) 

image_shape = clustering_data_metadata['utils']['image_shape']
background_mask = clustering_data_metadata['utils']['background_mask']
data_cluster1 = clustering_data_metadata['cluster_1']['data'] 
data_cluster2 = clustering_data_metadata['cluster_2']['data']
data_cluster3 = clustering_data_metadata['cluster_3']['data']
data_cluster4 = clustering_data_metadata['cluster_4']['data']
data_cluster5 = clustering_data_metadata['cluster_5']['data']
data_cluster6 = clustering_data_metadata['cluster_6']['data']
data_cluster7 = clustering_data_metadata['cluster_7']['data']
data_cluster8 = clustering_data_metadata['cluster_8']['data']

# Reference cluster labels
ref_labels = ([1]*data_cluster1.shape[1] + [2]*data_cluster2.shape[1] + [3]*data_cluster3.shape[1] + [4]*data_cluster4.shape[1] + [5]*data_cluster5.shape[1] +
              [6]*data_cluster6.shape[1] + [7]*data_cluster7.shape[1] + [8]*data_cluster8.shape[1])

In [6]:
# Add off-tissue pixels to each ion image
dataset = np.hstack((data_cluster1, data_cluster2, data_cluster3, data_cluster4, data_cluster5, data_cluster6, data_cluster7, data_cluster8))
total_num_pixels = np.prod(image_shape)
num_mz_bins = dataset.shape[1]

dataset_final = np.zeros((total_num_pixels, num_mz_bins))
for mz_index in range(num_mz_bins):
    ion_image_flat = dataset[:, mz_index]
    ion_image = make_ion_image(ion_image_flat, image_shape, background_mask, fill_zeros=True, show_image=False)
    dataset_final[:, mz_index] = ion_image.flatten()

# Save to text file for DeepION
np.savetxt(r'C:\Users\Leo\Documents\Second_paper\Final_2nd_paper\Final_final\Comparisons\Colocalization\DeepION\Data\zebra_fish_for_DeepION_matrix.txt', dataset_final)
del dataset, dataset_final

In [7]:
# Mass-to-charge ratios of the molecular species in each cluster
mz_per_cluster = {}
mz_per_cluster[1] = clustering_data_metadata['cluster_1']['mz'].tolist()
mz_per_cluster[2] = clustering_data_metadata['cluster_2']['mz'].tolist()
mz_per_cluster[3] = clustering_data_metadata['cluster_3']['mz'].tolist()
mz_per_cluster[4] = clustering_data_metadata['cluster_4']['mz'].tolist()
mz_per_cluster[5] = clustering_data_metadata['cluster_5']['mz'].tolist()
mz_per_cluster[6] = clustering_data_metadata['cluster_6']['mz'].tolist()
mz_per_cluster[7] = clustering_data_metadata['cluster_7']['mz'].tolist()
mz_per_cluster[8] = clustering_data_metadata['cluster_8']['mz'].tolist()

mz_list = []
for cluster_label in range(1, 8): 
    mz_list.append(mz_per_cluster[cluster_label])

mz_list = np.array(list(chain(*mz_list)))

# Save as a CSV file for DeepION
np.savetxt(r'C:\Users\Leo\Documents\Second_paper\Final_2nd_paper\Final_final\Comparisons\Colocalization\DeepION\Data\zebra_fish_for_DeepION_peaks.csv', mz_list, delimiter=",", fmt="%d", comments='')

#### DeepION clustering workflow

In [8]:
input_matrix = r'C:\Users\Leo\Documents\Second_paper\Final_2nd_paper\Final_final\Comparisons\Colocalization\DeepION\Data\zebra_fish_for_DeepION_matrix.txt'
input_peak_list = r'C:\Users\Leo\Documents\Second_paper\Final_2nd_paper\Final_final\Comparisons\Colocalization\DeepION\Data\zebra_fish_for_DeepION_peaks.csv'
input_shape = image_shape # (height, width)
mode = "COL"  # "COL" for colocalization
ion_mode = "positive"
num_co_localized = 10

In [9]:
start_time = time.time()

In [10]:
train_deepion(input_matrix, input_shape, mode)
features = predict_deepion(input_matrix, input_shape, mode)

Starting DeepION training...
Step 1: Start DeepION training ...


C:\Users\Leo\anaconda3\envs\spatial_IMS_env_compare\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Leo\anaconda3\envs\spatial_IMS_env_compare\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch 0 : batch : 0 loss: 4.1373
epoch 1 : batch : 0 loss: 2.0272
epoch 2 : batch : 0 loss: 1.7821
epoch 3 : batch : 0 loss: 1.6976
epoch 4 : batch : 0 loss: 1.6557
epoch 5 : batch : 0 loss: 1.6060
epoch 6 : batch : 0 loss: 1.6186
epoch 7 : batch : 0 loss: 1.6166
epoch 8 : batch : 0 loss: 1.5522
epoch 9 : batch : 0 loss: 1.6005
epoch 10 : batch : 0 loss: 1.5259
epoch 11 : batch : 0 loss: 1.5008
epoch 12 : batch : 0 loss: 1.4852
epoch 13 : batch : 0 loss: 1.4512
epoch 14 : batch : 0 loss: 1.4085
epoch 15 : batch : 0 loss: 1.4504
epoch 16 : batch : 0 loss: 1.3140
epoch 17 : batch : 0 loss: 1.2945
epoch 18 : batch : 0 loss: 1.2714
epoch 19 : batch : 0 loss: 1.2325
epoch 20 : batch : 0 loss: 1.1882
epoch 21 : batch : 0 loss: 1.2191
epoch 22 : batch : 0 loss: 1.1349
epoch 23 : batch : 0 loss: 1.1509
epoch 24 : batch : 0 loss: 1.0636
epoch 25 : batch : 0 loss: 1.0175
epoch 26 : batch : 0 loss: 0.9804
epoch 27 : batch : 0 loss: 0.8786
epoch 28 : batch : 0 loss: 0.8644
epoch 29 : batch : 0 los

C:\Users\Leo\anaconda3\envs\spatial_IMS_env_compare\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Leo\anaconda3\envs\spatial_IMS_env_compare\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Feature extraction completed!


In [11]:
# Dimensionality reduction with UMAP
features_umap = reduce_dimensionality(features)

Applying dimensionality reduction...


C:\Users\Leo\anaconda3\envs\spatial_IMS_env_compare\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Dimensionality reduction completed!


In [12]:
# HDBSCAN clustering
HDBSCAN_model = HDBSCAN(min_cluster_size=5, max_cluster_size=50, metric="euclidean", n_jobs=-1)
HDBSCAN_model.fit(features_umap)
DeepION_labels = HDBSCAN_model.labels_

In [13]:
end_time = time.time()
compute_time = end_time - start_time
print(f"DeepION execution time: {compute_time:.6f} seconds")

DeepION execution time: 4651.384867 seconds


In [14]:
# Clustering performance 
ARI = np.round(adjusted_rand_score(ref_labels, DeepION_labels), 4)
print('Adjusted Rand index:', ARI)
AMI = np.round(adjusted_mutual_info_score(ref_labels, DeepION_labels, average_method='arithmetic'), 4)
print('Adjusted mutual information:', AMI)
BARI = np.round(balanced_adjusted_rand_index(np.array(ref_labels), np.array(DeepION_labels)), 4)
print('Balanced adjusted Rand index:', BARI)
balanced_homogeneity, balanced_completeness, balanced_v_measure = np.round(balanced_homogeneity_completeness_v_measure(np.array(ref_labels), np.array(DeepION_labels)), 4)
print('Balanced V-measure:', balanced_v_measure)

Adjusted Rand index: 0.5589
Adjusted mutual information: 0.7472
Balanced adjusted Rand index: 0.7598
Balanced V-measure: 0.8661
